In [ ]:
#build and push docker images for latest script change
!source .env.sh
!bash scripts/build_images.sh
#!bash scripts/push_images.sh

In [1]:
#start rafiki
!bash ./scripts/start.sh

---------------------------------------
| Creating Docker swarm for Rafiki... |
---------------------------------------
Swarm initialized: current node (xtpj1f9r4qju8nig91r01z99x) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-3iauyez8ohulb9z17jhi3fycs7p93v85nzbb4bpia62bvz5krk-0izj0j6eiut5owus1ab58cup2 127.0.0.1:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

37cme00wt6irg7hpkeuhpclsg
------------------------------------
| Creating Rafiki's logs folder... |
------------------------------------
Created Rafiki's logs directory!
------------------------------------------------
| Pulling images for Rafiki from Docker Hub... |
------------------------------------------------
Pulling images required by Rafiki from Docker Hub...
postgres:10.5 already exists locally
redis:5.0-rc already exists locally
rafikiai/rafiki_admin:0.0.5 already exists locally
rafikiai/rafiki_

In [ ]:
#stop rafiki
!bash ./scripts/stop.sh

In [2]:
#start client
from rafiki.client import Client
client = Client(admin_host='localhost', admin_port=8000)
client.login(email='superadmin@rafiki', password='rafiki')

{'user_id': 'f331b6a6-8be0-4b96-8ccd-4fe8da73abab', 'user_type': 'SUPERADMIN'}

In [3]:
#add model for FEATURE_VECTOR_CLASSIFICATION
'''
client.create_model(
    name='SkDtVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkDt.py',
    model_class='SkDtVector'
)


client.create_model(
    name='TfSingleHiddenLayerVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/TfSingleHiddenLayer.py',
    model_class='TfSingleHiddenLayerVector'
)
'''

client.create_model(
    name='SkSvmVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkSvm.py',
    model_class='SkSvmVector'
)

client.create_model(
    name='SkRfVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkRf.py',
    model_class='SkRfVector'
)

client.create_model(
    name='SkLogVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkLog.py',
    model_class='SkLogVector'
)

#check loaded models by task
client.get_models_of_task(task='FEATURE_VECTOR_CLASSIFICATION')

[{'datetime_created': 'Mon, 29 Oct 2018 04:51:45 GMT',
  'docker_image': 'rafikiai/rafiki_worker:0.0.5',
  'model_class': 'SkRfVector',
  'name': 'SkRfVector',
  'task': 'FEATURE_VECTOR_CLASSIFICATION',
  'user_id': 'f331b6a6-8be0-4b96-8ccd-4fe8da73abab'},
 {'datetime_created': 'Mon, 29 Oct 2018 04:51:45 GMT',
  'docker_image': 'rafikiai/rafiki_worker:0.0.5',
  'model_class': 'SkLogVector',
  'name': 'SkLogVector',
  'task': 'FEATURE_VECTOR_CLASSIFICATION',
  'user_id': 'f331b6a6-8be0-4b96-8ccd-4fe8da73abab'}]

In [12]:
#create a training job
client.create_train_job(
    app='concept_drift',
    task='FEATURE_VECTOR_CLASSIFICATION',
    train_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true',
    test_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_test.zip?raw=true',
    budget_type='MODEL_TRIAL_COUNT',
    budget_amount=2
)

{'app': 'concept_drift',
 'app_version': 2,
 'id': 'dc961cb6-d4e7-4035-be32-9f46093d43ab'}

In [15]:
#get train job status
client.get_train_job(app='concept_drift')

{'app': 'concept_drift',
 'app_version': 2,
 'budget_amount': 2,
 'budget_type': 'MODEL_TRIAL_COUNT',
 'datetime_completed': 'Mon, 29 Oct 2018 04:56:07 GMT',
 'datetime_started': 'Mon, 29 Oct 2018 04:55:54 GMT',
 'id': 'dc961cb6-d4e7-4035-be32-9f46093d43ab',
 'status': 'COMPLETED',
 'task': 'FEATURE_VECTOR_CLASSIFICATION',
 'test_dataset_uri': 'https://github.com/sdragon007/sea_dataset/blob/master/sea_test.zip?raw=true',
 'train_dataset_uri': 'https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true',
 'workers': [{'datetime_started': 'Mon, 29 Oct 2018 04:55:54 GMT',
   'datetime_stopped': 'Mon, 29 Oct 2018 04:56:07 GMT',
   'model_name': 'SkRfVector',
   'replicas': 2,
   'service_id': 'da995c58-84e8-40c3-9332-e33778b7b5a2',
   'status': 'STOPPED'},
  {'datetime_started': 'Mon, 29 Oct 2018 04:55:54 GMT',
   'datetime_stopped': 'Mon, 29 Oct 2018 04:56:04 GMT',
   'model_name': 'SkLogVector',
   'replicas': 2,
   'service_id': '06eff8f1-cdf4-470f-bed0-e3365f93d036',


In [14]:
#get job metrics
client.get_best_trials_of_train_job(app='concept_drift')

[{'datetime_started': 'Mon, 29 Oct 2018 04:56:02 GMT',
  'datetime_stopped': 'Mon, 29 Oct 2018 04:56:05 GMT',
  'id': '67822bf7-4915-4861-8b6d-11edeb8751b2',
  'knobs': {'C': 0.2921613107220019,
   'max_iter': 127,
   'penalty': 'l2',
   'tol': 2.2290456039461248e-05},
  'model_name': 'SkLogVector',
  'score': 0.8955},
 {'datetime_started': 'Mon, 29 Oct 2018 04:55:59 GMT',
  'datetime_stopped': 'Mon, 29 Oct 2018 04:56:02 GMT',
  'id': 'f876cf41-8922-4a0f-af29-e52001e73213',
  'knobs': {'C': 30.907091604492077,
   'max_iter': 116,
   'penalty': 'l2',
   'tol': 0.0009656477147308054},
  'model_name': 'SkLogVector',
  'score': 0.8935},
 {'datetime_started': 'Mon, 29 Oct 2018 04:56:01 GMT',
  'datetime_stopped': 'Mon, 29 Oct 2018 04:56:04 GMT',
  'id': '6b8f677f-a90e-4110-8de2-02ca3f4db3ef',
  'knobs': {'C': 0.12222884594099637,
   'max_iter': 17,
   'penalty': 'l2',
   'tol': 6.379075048113337e-05},
  'model_name': 'SkLogVector',
  'score': 0.893}]

In [ ]:
client.stop_train_job(app = 'concept_drift')

In [ ]:
#startinference job
client.create_inference_job(app='concept_drift')

In [ ]:
#stop inference job
client.stop_inference_job(app='concept_drift')